In [3]:
from manim import *

[04/22/24 20:23:45] WARNING  Missing Plugins: {''}                                                   ]8;id=157897;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py\__init__.py]8;;\:]8;id=758203;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py#17\17]8;;\

In [5]:
%%manim -ql --fps 30 -v WARNING LayeredGraphProjection

import networkx as nx
import numpy as np

class LayeredGraphProjection(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=135 * DEGREES)

        # Create a graph with layers
        G = nx.Graph()
        layers = {1: 0, 2: 1, 3: 1, 4: 2, 5: 2, 6: 0}
        edges = [(1, 2), (1, 3), (3, 4), (2, 4), (2, 5), (4, 5), (6, 5)]
        G.add_edges_from(edges)

        # Define colors for each layer
        layer_colors = {
            0: BLUE,
            1: GREEN,
            2: RED
        }

        # 2D layout for base projection
        pos = nx.spring_layout(G, dim=2)
        z_scale = 1  # Scale for the z-axis to visually separate layers more clearly
        pos_3d = {node: np.array([*pos[node], layers[node] * z_scale]) for node in G.nodes}

        # Create 3D nodes and edges
        nodes_3d = [Sphere(radius=0.1, color=layer_colors[layers[node]]).move_to(pos_3d[node]) for node in G.nodes]
        edges_3d = [Line3D(start=pos_3d[edge[0]], end=pos_3d[edge[1]], color=WHITE) for edge in G.edges]

        # Calculate minimum Z for the projection plane to sit below the lowest node
        min_z = min(layers.values()) * z_scale - 1  # One unit below the lowest layer

        # Maximum layer for scaling purposes
        max_layer = max(layers.values())

        # Create 2D nodes and edges with perspective scaling
        nodes_2d = [Dot(point=[*pos[node], min_z], color=layer_colors[layers[node]], radius=0.1 * (1 + (layers[node] / (max_layer + 1))))
                    for node in G.nodes]
        edges_2d = [Line(start=[*pos[edge[0]], min_z], end=[*pos[edge[1]], min_z], color=LIGHT_GRAY) for edge in G.edges]

        # Projection lines connecting 3D and 2D nodes
        projection_lines = [DashedLine(start=pos_3d[node], end=[*pos[node], min_z], color=GRAY) for node in G.nodes]

        # Create a transparent square surface for the 2D projection plane
        plane_size = 3.5  # Size of each plane, adjust as necessary
        projection_plane = Rectangle(width=plane_size, height=plane_size, fill_color=BLUE, fill_opacity=0.5, stroke_width=0)
        projection_plane.shift(OUT * min_z)  # Positioned one step below the lowest node

        # Create planes for each layer
        unique_layers = set(layers.values())
        layer_planes = []
        for layer in unique_layers:
            plane = Rectangle(width=plane_size, height=plane_size, fill_color=layer_colors[layer], fill_opacity=0.2, stroke_width=0)
            plane.shift(OUT * layer * z_scale)  # Offset each plane to match its respective layer height
            layer_planes.append(plane)

        # Group all elements including the planes
        graph_group = Group(*projection_plane, *layer_planes, *nodes_3d, *edges_3d, *nodes_2d, *edges_2d, *projection_lines)

        # Add the entire graph and projection in one step
        #self.play(FadeIn(graph_group), run_time=1)
        self.add(graph_group)

        # Camera swing around the scene
        self.move_camera(phi=80 * DEGREES, theta=-90 * DEGREES, run_time=1)

        #self.wait(1)  # Short wait time to view the complete graph and projection after the camera swing

# Render with a lower frame rate if necessary: manim -pql --fps 30 script.py


Manim Community v0.18.0